# CVLR Count

## Basic Packages

In [59]:
import os
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np
from os.path import exists
import math

# Current working directory
basepath = Path.cwd().parent.parent.parent

# Read list of stations within model coverage area
raw_data = os.path.join(basepath, '01 Raw Data' )
# raw_data

## Read CVLR turning count data

In [97]:
cvlr_count_filenames  = pd.read_csv(f'{raw_data}/20 CVLR count data/CVLR Count Data/cvlr_count_location_filenames.csv')
cvlr_count_filenames = cvlr_count_filenames[['Site No','Workbook Name','Arms']].copy()
cvlr_count_filenames.head()

,Site No,Workbook Name,Arms
0,1,"1. Warwick Rd, Greyfriars Rd.xls.xlsx",3
1,2,"2. Queen Victoria Rd,Croft Rd.xls.xlsx",3
2,3,"3. Corporation Str,Fleet Str.xls.xlsx",3
3,4,"4. Corporation Str, Hill Str.xls.xlsx",3
4,5,"5. Corporation Str,Upper Well S.xls.xlsx",3


## Individual Turn and Queue Data

### Turning Count Data

In [179]:
### Collecting Turns data for all sites
site_df = pd.DataFrame(columns=['siteno','site','from','to','period','total'])
turn_df = pd.DataFrame(columns=['siteno','site','from','to','period','total'])
for index, row in cvlr_count_filenames.iterrows():
    filename = row['Workbook Name']
    siteno = row['Site No']
    arms = row['Arms']
    sitename = filename.split(".xls.xlsx")[0]
    # print(filename)
    interim_file = pd.read_excel(f'{raw_data}/20 CVLR count data/CVLR Count Data/{filename}',
                             sheet_name=sitename,
                             skiprows=20,
                             header=None)

    ### Looping through different entry arms
    for i in range(0,arms):
        x = 52*i
        from_arm = interim_file.loc[x,0].split(": ")[1]

        ### Looping through different exit arms
        for j in range(0,arms):
            y = 9 * j
            extracts = interim_file.iloc[x + 3:x + 51, y:y + 9].copy()  # Make a copy
            extracts_header = interim_file.iloc[x + 2,y:y + 9]

            extracts.columns = extracts_header
            extracts['siteno'] = siteno
            extracts['site'] = sitename
             
            extracts['from'] = from_arm  
            extracts['to'] = interim_file.loc[x + 1, y].split(": ")[1]

            extracts_final = extracts[['siteno','site','from', 'to', 'Period', 'Total']].copy()
            extracts_final.rename(columns={'Period': 'period', 'Total': 'total'}, inplace=True)
            site_df = pd.concat([site_df, extracts_final])
            arm_list = site_df['to'].unique().tolist()
turn_df = pd.concat([turn_df, site_df])    
turn_df['period'] = pd.to_datetime(turn_df['period'], format='%H:%M:%S')           
turn_df = turn_df.reset_index(drop=True)

# Create a function to categorize the time periods
def categorize_time_period(time):
    if pd.Timestamp('08:00').time() <= time < pd.Timestamp('09:00').time():
        return 'AM'
    ### For operational purpose, using IP peak hour with max totals 15:00-16:00
    ### However, the approach is to ignore IP for now unless requested
    elif pd.Timestamp('15:00').time() <= time < pd.Timestamp('16:00').time():   
        return 'IP'
    elif pd.Timestamp('17:00').time() <= time < pd.Timestamp('18:00').time():
        return 'PM'
    else:
        return 'OP'


# Apply the function to create the 'time_period' column
turn_df['peak'] = pd.to_datetime(turn_df['period'], format='%H:%M:%S').dt.time.apply(categorize_time_period)
turn_df['period'] = turn_df['period'].dt.strftime('%H:%M')
turn_df.head()


,siteno,site,from,to,period,total,peak
0,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,Warwick Road ne,07:00,0,OP
1,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,Warwick Road ne,07:15,1,OP
2,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,Warwick Road ne,07:30,0,OP
3,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,Warwick Road ne,07:45,0,OP
4,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,Warwick Road ne,08:00,5,AM


In [159]:
### Filter to remove OP and add up totals for AM, IP and PM

filtered_turn_df = turn_df[(turn_df['peak']!='OP') & (turn_df['peak']!='IP')].copy()

peak_totals = filtered_turn_df[['siteno','site','from', 'to','peak','total']].groupby(['siteno','site','from', 'to','peak']).sum().reset_index()
peak_totals['period'] = peak_totals['peak']
peak_totals.head()

filtered_turn_df = pd.concat([filtered_turn_df, peak_totals])
filtered_turn_df.sort_values(['siteno','site','from', 'to'],inplace=True)
filtered_turn_df.tail()

,siteno,site,from,to,period,total,peak
11033,20,"20. Nicholls Str,Thackhall Str",Thackhall Street,Thackhall Street,17:15,0,PM
11034,20,"20. Nicholls Str,Thackhall Str",Thackhall Street,Thackhall Street,17:30,0,PM
11035,20,"20. Nicholls Str,Thackhall Str",Thackhall Street,Thackhall Street,17:45,0,PM
466,20,"20. Nicholls Str,Thackhall Str",Thackhall Street,Thackhall Street,AM,0,AM
467,20,"20. Nicholls Str,Thackhall Str",Thackhall Street,Thackhall Street,PM,0,PM


In [160]:
pivot_turn_df = filtered_turn_df.pivot(index=['siteno','site','from', 'to'], columns=['period'],values='total').reset_index()
pivot_turn_df.head()

period,siteno,site,from,to,08:00,08:15,08:30,08:45,17:00,17:15,17:30,17:45,AM,PM
0,1,"1. Warwick Rd, Greyfriars Rd",Greyfriars Road,Greyfriars Road,1,4,4,3,3,3,5,4,12,15
1,1,"1. Warwick Rd, Greyfriars Rd",Greyfriars Road,Warwick Road ne,26,29,30,33,34,37,34,44,118,149
2,1,"1. Warwick Rd, Greyfriars Rd",Greyfriars Road,Warwick Road sw,4,9,4,8,15,7,8,10,25,40
3,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,Greyfriars Road,18,21,33,44,56,48,55,33,116,192
4,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,Warwick Road ne,5,2,2,0,4,7,3,2,9,16


In [161]:
pivot_turn_df.to_csv(f'{basepath}/03 Output/11 CVLR/turning_counts_v1.csv')

### Queue Data

In [200]:
filename = '1. Warwick Rd, Greyfriars Rd.xls.xlsx'
sitename = filename.split(".xls.xlsx")[0]
arms = 3
q_interim_file = pd.read_excel(f'{raw_data}/20 CVLR count data/CVLR Count Data/{filename}',
                            sheet_name='Sheet2',
                            skiprows=1,
                            header=None)
extracts_q = q_interim_file.iloc[:, :arms+3].copy()

extracts_header = ['period','x-','to_time','arm_1','arm_2','arm_3']

extracts_q.columns = extracts_header
extracts_q['site'] = sitename
extracts_q['siteno'] = 1
extracts_q.head()

def categorize_15_min_period(time):
    if pd.Timestamp('07:00').time() <= time < pd.Timestamp('07:15').time():
        return '07:00'
    elif pd.Timestamp('07:15').time() <= time < pd.Timestamp('07:30').time():
        return '07:15'
    elif pd.Timestamp('07:30').time() <= time < pd.Timestamp('07:45').time():
        return '07:30'
    elif pd.Timestamp('07:45').time() <= time < pd.Timestamp('08:00').time():
        return '07:45'
    ### For operational purpose, using IP peak hour with max totals 15:00-16:00
    ### However, the approach is to ignore IP for now unless requested
    # elif pd.Timestamp('15:00').time() <= time < pd.Timestamp('15:15').time():
    #     return '15:00'
    # elif pd.Timestamp('15:15').time() <= time < pd.Timestamp('15:30').time():
    #     return '15:15'
    # elif pd.Timestamp('15:30').time() <= time < pd.Timestamp('15:45').time():
    #     return '15:30'
    # elif pd.Timestamp('15:45').time() <= time < pd.Timestamp('16:00').time():
    #     return '15:45'
    elif pd.Timestamp('17:00').time() <= time < pd.Timestamp('17:15').time():
        return '17:00'
    elif pd.Timestamp('17:15').time() <= time < pd.Timestamp('17:30').time():
        return '17:15'
    elif pd.Timestamp('17:30').time() <= time < pd.Timestamp('17:45').time():
        return '17:30'
    elif pd.Timestamp('17:45').time() <= time < pd.Timestamp('18:00').time():
        return '17:45'
    # else:
    #     return 'OP'
extracts_q.head()
extracts_q['period_15'] = pd.to_datetime(extracts_q['period'], format='%H:%M:%S').dt.time.apply(categorize_15_min_period)

grouped_15min = extracts_q[['siteno','site','period_15','arm_1','arm_2','arm_3']].groupby('period_15').max().reset_index()
grouped_15min.head()
# grouped_15min_melted = pd.melt(grouped_15min,  id_vars=['siteno','site','period_15'], var_name='arm', value_name='max_queue')
# grouped_15min_melted.head()
# pivot_15min_q = grouped_15min_melted.pivot(index=['siteno','site','arm'], columns=['period_15'],values='max_queue').reset_index()
# pivot_15min_q.head()

,period_15,siteno,site,arm_1,arm_2,arm_3
0,07:00,1,"1. Warwick Rd, Greyfriars Rd",2,1,0
1,07:15,1,"1. Warwick Rd, Greyfriars Rd",0,1,2
2,07:30,1,"1. Warwick Rd, Greyfriars Rd",1,2,4
3,07:45,1,"1. Warwick Rd, Greyfriars Rd",1,2,4
4,17:00,1,"1. Warwick Rd, Greyfriars Rd",6,8,3


## Combined Turn and Queue data

In [195]:
# Create a function to categorize the time periods
def categorize_time_period(time):
    if pd.Timestamp('08:00').time() <= time < pd.Timestamp('09:00').time():
        return 'AM'
    ### For operational purpose, using IP peak hour with max totals 15:00-16:00
    ### However, the approach is to ignore IP for now unless requested
    elif pd.Timestamp('15:00').time() <= time < pd.Timestamp('16:00').time():   
        return 'IP'
    elif pd.Timestamp('17:00').time() <= time < pd.Timestamp('18:00').time():
        return 'PM'
    else:
        return 'OP'

def categorize_15_min_period(time):
    if pd.Timestamp('07:00').time() <= time < pd.Timestamp('07:15').time():
        return '07:00:00'
    elif pd.Timestamp('07:15').time() <= time < pd.Timestamp('07:2').time():
        return '07:15:00'
    elif pd.Timestamp('07:30').time() <= time < pd.Timestamp('07:45').time():
        return '07:30:00'
    elif pd.Timestamp('07:45').time() <= time < pd.Timestamp('08:00').time():
        return '07:45:00'
    elif pd.Timestamp('17:00').time() <= time < pd.Timestamp('17:15').time():
        return '17:00:00'
    elif pd.Timestamp('17:15').time() <= time < pd.Timestamp('17:30').time():
        return '17:15:00'
    elif pd.Timestamp('17:30').time() <= time < pd.Timestamp('17:45').time():
        return '17:30:00'
    elif pd.Timestamp('17:45').time() <= time < pd.Timestamp('18:00').time():
        return '17:45:00'
    else:
        return '01:00:00'
    


In [226]:
### Collecting Turns data for all sites

turn_df = pd.DataFrame(columns=['siteno','site','from','to','period','total'])
queue_df = pd.DataFrame(columns=['siteno','site','arm','period_15','max_queue'])


for index, row in cvlr_count_filenames.iterrows():
    filename = row['Workbook Name']
    siteno = row['Site No']
    arms = row['Arms']
    sitename = filename.split(".xls.xlsx")[0]
    # print(filename)
    interim_file = pd.read_excel(f'{raw_data}/20 CVLR count data/CVLR Count Data/{filename}',
                             sheet_name=sitename,
                             skiprows=20,
                             header=None)
    site_df = pd.DataFrame(columns=['siteno','site','from','to','period','total'])
    ### Looping through different entry arms
    for i in range(0,arms):
        x = 52*i
        from_arm = interim_file.loc[x,0].split(": ")[1]

        ### Looping through different exit arms
        for j in range(0,arms):
            y = 9 * j
            extracts = interim_file.iloc[x + 3:x + 51, y:y + 9].copy()  # Make a copy
            extracts_header = interim_file.iloc[x + 2,y:y + 9]

            extracts.columns = extracts_header
            extracts['siteno'] = siteno
            extracts['site'] = sitename
             
            extracts['from'] = from_arm  
            extracts['to'] = interim_file.loc[x + 1, y].split(": ")[1]

            extracts_final = extracts[['siteno','site','from', 'to', 'Period', 'Total']].copy()
            extracts_final.rename(columns={'Period': 'period', 'Total': 'total'}, inplace=True)
            site_df = pd.concat([site_df, extracts_final])
            arm_list = site_df['from'].unique().tolist()

    turn_df = pd.concat([turn_df, site_df])
    q_interim_file = pd.read_excel(f'{raw_data}/20 CVLR count data/CVLR Count Data/{filename}',
                                sheet_name='Sheet2',
                                skiprows=1,
                                header=None)
    extracts_q = q_interim_file.iloc[:, :arms+3].copy()
    extracts_q_header = ['period','x-','to_time',*arm_list]
    # print(extracts_q_header)
    extracts_q.columns = extracts_q_header
    extracts_q['site'] = sitename
    extracts_q['siteno'] = siteno

    extracts_q['period_15'] = pd.to_datetime(extracts_q['period'], format='%H:%M:%S').dt.time.apply(categorize_15_min_period)
    grouped_15min = extracts_q[['siteno','site','period_15',*arm_list]].groupby('period_15').max().reset_index()
    site_q_df = pd.melt(grouped_15min,  id_vars=['siteno','site','period_15'], var_name='arm', value_name='max_queue')

    queue_df = pd.concat([queue_df, site_q_df])

        
turn_df['period'] = pd.to_datetime(turn_df['period'], format='%H:%M:%S')           
turn_df = turn_df.reset_index(drop=True)
# Apply the function to create the 'time_period' column
turn_df['peak'] = pd.to_datetime(turn_df['period'], format='%H:%M:%S').dt.time.apply(categorize_time_period)
turn_df['period'] = turn_df['period'].dt.strftime('%H:%M')




In [227]:
turn_df.head()

,siteno,site,from,to,period,total,peak
0,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,Warwick Road ne,07:00,0,OP
1,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,Warwick Road ne,07:15,1,OP
2,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,Warwick Road ne,07:30,0,OP
3,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,Warwick Road ne,07:45,0,OP
4,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,Warwick Road ne,08:00,5,AM


In [228]:
queue_df.head()

,siteno,site,arm,period_15,max_queue
0,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,07:00,2
1,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,07:15,0
2,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,07:30,1
3,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,07:45,1
4,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,17:00,6


In [229]:
### Filter to remove OP and add up totals for AM, IP and PM

filtered_turn_df = turn_df[(turn_df['peak']!='OP') & (turn_df['peak']!='IP')].copy()

peak_totals = filtered_turn_df[['siteno','site','from', 'to','peak','total']].groupby(['siteno','site','from', 'to','peak']).sum().reset_index()
peak_totals['period'] = peak_totals['peak']

filtered_turn_df = pd.concat([filtered_turn_df, peak_totals])
filtered_turn_df.sort_values(['siteno','site','from', 'to'],inplace=True)
pivot_turn_df = filtered_turn_df.pivot(index=['siteno','site','from', 'to'], columns=['period'],values='total').reset_index()
pivot_turn_df.head()

period,siteno,site,from,to,08:00,08:15,08:30,08:45,17:00,17:15,17:30,17:45,AM,PM
0,1,"1. Warwick Rd, Greyfriars Rd",Greyfriars Road,Greyfriars Road,1,4,4,3,3,3,5,4,12,15
1,1,"1. Warwick Rd, Greyfriars Rd",Greyfriars Road,Warwick Road ne,26,29,30,33,34,37,34,44,118,149
2,1,"1. Warwick Rd, Greyfriars Rd",Greyfriars Road,Warwick Road sw,4,9,4,8,15,7,8,10,25,40
3,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,Greyfriars Road,18,21,33,44,56,48,55,33,116,192
4,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,Warwick Road ne,5,2,2,0,4,7,3,2,9,16


In [231]:
pivot_turn_df.to_csv(f'{basepath}/03 Output/11 CVLR/turning_counts_v2.csv')

In [230]:
### Queue in proper format
pivot_q = queue_df.pivot(index=['siteno','site','arm'], columns=['period_15'],values='max_queue').reset_index()
pivot_q.head()

period_15,siteno,site,arm,07:00,07:15,07:30,07:45,17:00,17:15,17:30,17:45
0,1,"1. Warwick Rd, Greyfriars Rd",Greyfriars Road,0,2,4,4,3,3,2,3
1,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road ne,2,0,1,1,6,5,3,4
2,1,"1. Warwick Rd, Greyfriars Rd",Warwick Road sw,1,1,2,2,8,21,3,6
3,2,"2. Queen Victoria Rd,Croft Rd",Croft Road,1,3,3,2,1,3,5,3
4,2,"2. Queen Victoria Rd,Croft Rd",Queen Victoria Road north,0,0,2,3,3,4,3,2


In [232]:
pivot_q.to_csv(f'{basepath}/03 Output/11 CVLR/queue_df_v2.csv')